## Installation


Global python env


First, to install PSG in python, we need to navigate to Aorda install directory and install wheel file directly. Additionally, we may want to install Gurobi as well (particularly for mixed integer programming). This requires obtaining a key from Gurobi, then simply doing a `pip install gurobipy`. If you don't set the key for Gurobi, we won't get a clear warning when we run PSG with Gurobi partially installed, it simply won't give correct results. 


In [2]:
%pip install C:\Aorda\PSG\Python\psgpython-3.2-cp38-cp38-win_amd64.whl

Processing c:\aorda\psg\python\psgpython-3.2-cp38-cp38-win_amd64.whl
psgpython is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
os.add_dll_directory('C:\Aorda\PSG\lib')

<AddedDllDirectory('C:\\Aorda\\PSG\\lib')>

In [3]:
import psgpython as psg # import should work now, sometimes need to register dll once as on line below:


In this same folder should be a few example scripts that call psg in python. 

## Loading an example
The easiest way to start working with PSG is to download a case study with a pre-formatted problem statement along with data from Prof. Uryasev's [website](http://uryasev.ams.stonybrook.edu/index.php/research/testproblems/). 

Here an example where we download and unzip case study. Note that the data files in text format need to be in the same directory as the problem statement:

In [4]:
import requests
import zipfile
import io

In [5]:
prob = requests.get('http://uryasev.ams.stonybrook.edu/wp-content/uploads/2019/05/problem_hmm_discrete.zip')
uz = zipfile.ZipFile(io.BytesIO(prob.content))
uz.extractall("./psg_example_hmm/")

In [6]:
psg_prob = psg.psg_importfromtext('./psg_example_hmm/problem_hmm_discrete.txt')

OK. Problem Imported



Here is an important detail. When importing a problem statement, PSG will by default split it into a list of strings. For PSG to work correctly, we need to join this list with newline characters between each statement.

In [7]:
psg_prob['problem_statement'] = '\n'.join(psg_prob['problem_statement'])

In [8]:
psg.psg_verify(psg_prob) # check problem - mostly useful when testing as the actual solver will cause a kernel crash if problem is not formatted correctly

Running solver
Reading problem formulation
Asking for data information
Getting data
100% of vector_observation_300 was read
The problem is verified.
Solver finished verification.
It's OK.

'Verified!'

In [17]:
psg.psg_solver(psg_prob)

Running solver
Reading problem formulation
Asking for data information
Getting data
100% of matrix_sum_of_x_5 was read
100% of matrix_10000_1 was read
100% of matrix_10000_2 was read
100% of matrix_10000_3 was read
100% of matrix_10000_4 was read
100% of matrix_10000_5 was read
Start optimization
Ext.iteration=0  Objective=0.000000000000E+00  Residual=0.211547412342E+01
Ext.iteration=66  Objective=0.925128338193E+00  Residual=0.131829585682E-13
Start stage  1
Ext.iteration=0  Objective=0.925128338193E+00  Residual=-.111793664072E-01
Ext.iteration=44  Objective=0.919200709398E+00  Residual=0.598771736926E-06
Start stage  2
Ext.iteration=0  Objective=0.919200709398E+00  Residual=-.514913516569E-03
Ext.iteration=8  Objective=0.919200709398E+00  Residual=-.514913516569E-03
Start stage  3
Ext.iteration=0  Objective=0.919200709398E+00  Residual=-.385433335677E-03
Ext.iteration=8  Objective=0.919200709398E+00  Residual=-.385433335677E-03
Start stage  4
Ext.iteration=0  Objective=0.91920070939

{'problem_name': 'problem_1',
 'solution_status': 'optimal',
 'problem_statement': ['minimize',
  '  linear(matrix_sum_of_x_5)',
  'Constraint: <= 0.0281',
  '  prmulti_pen(0,matrix_10000_1,matrix_10000_2,matrix_10000_3,matrix_10000_4,matrix_10000_5)',
  'Box: >= 0, <= 1'],
 'output': ['Problem: problem_1, solution_status = optimal',
  'Timing: data_loading_time = 0.25, preprocessing_time = 0.01, solving_time = 0.17',
  'Variables: optimal_point = point_problem_1',
  'Objective: objective = 0.918316820975 [2.266664024378E-03]',
  'Constraint: constraint_1 =  2.809999805748E-02 [ 2.810001055748E-02]',
  'Function: linear(matrix_sum_of_x_5) =  9.183168209747E-01',
  'Function: prmulti_pen(0,matrix_10000_1,matrix_10000_2,matrix_10000_3,matrix_10000_4,matrix_10000_5) =  2.809999805748E-02'],
 'point_constraints_problem_1': [['constraint_1'], array([0.0281])],
 'point_slack_constraints_problem_1': [['constraint_1'], array([0.02810001])],
 'point_problem_1': [['attachment_point_1',
   'attac

## Defining a problem explicitly
This is one of the example scripts from the /Aorda/PSG/Python/Examples directory

In [18]:
import numpy as np

In [20]:

matrix_scenarios_body = np.reshape(np.array([1,4,8,3, 7,5,4,6, 2,8,1,0, 0,3,4,9],dtype='float64'),(4,4))
scenario_benchmark = np.array([0.2, 0.11, 0.6, 0.1])
matrix_scenarios_body = np.column_stack((matrix_scenarios_body, scenario_benchmark))
header = ["x%d" % (i) for i in range(1,matrix_scenarios_body.shape[1])]
matrix_scenarios = [header+['scenario_benchmark'], matrix_scenarios_body]
matrix_budget_body =np.array([1, 1, 1, 1],dtype='float64')
matrix_budget = [header, matrix_budget_body]
point_lowerbounds = [header, np.array([0, 0, 0, 0],dtype='float64')]

alpha = 0.95
allowExternal = True
suppressMessages = False
problem_name = "problem_test"
problem_statement = "minimize\n\
cvar_risk(%f,matrix_scenarios)\n\
Constraint: >= 4.5\n\
avg_g(matrix_scenarios)\n\
Constraint: == 1\n\
linear(matrix_budget)\n\
Box: >= point_lowerbounds\n\
Solver: van" % alpha
problem_dictionary = {'problem_name':problem_name, 'problem_statement':problem_statement, 'matrix_scenarios':matrix_scenarios,\
'matrix_budget':matrix_budget, 'point_lowerbounds':point_lowerbounds}

res = psg.psg_solver(problem_dictionary, allowExternal, suppressMessages)

print(res)


Running solver
Reading problem formulation
Asking for data information
Getting data
     20.0% of scenarios is processed
100% of matrix_scenarios was read
100% of matrix_budget was read
Start optimization
Ext.iteration=0  Objective=0.600000000000E+00  Residual=0.000000000000E+00
Ext.iteration=5  Objective=-.435000000000E+01  Residual=0.000000000000E+00
Optimization is stopped
Solution is optimal
Calculating resulting outputs. Writing solution.
Objective: objective = -4.35000000000 [-8.881784197001E-16]
Solver has normally finished. Solution was saved.
Problem: problem_1, solution_status = optimal
Timing: data_loading_time = 0.09, preprocessing_time = 0.01, solving_time = 0.01
Variables: optimal_point = point_problem_1
Objective: objective = -4.35000000000 [-8.881784197001E-16]
Constraint: constraint_1 =  4.500000000000E+00 [ 8.881784197001E-16]
Constraint: constraint_2 =  1.000000000000E+00 [ 0.000000000000E+00]
Function: cvar_risk(0.95,matrix_scenarios) = -4.350000000000E+00
Function:

In [22]:
# The solution is contained under the key 'point_problem_1'
res['point_problem_1']

[['x1', 'x2', 'x3', 'x4'], array([0.  , 0.6 , 0.19, 0.21])]

When loading or using your own data matrix as a psg matrix object, it should *always* be formatted as a c-style array (row major). If you pass in a matrix in column major order, sometimes PSG will simply crash, other times it will quietly output as incorrect solution.
You can use `np.asarray(..., order='C')` to convert your data and `.flags['C_

In [24]:
a = np.array([1.0,2.0])

In [27]:
a.flags['C_CONTIGUOUS']

True